sorce from https://dumps.wikimedia.org/simplewiki/20200301/

For a LDA model of Wikipedia we have these steps
- Data Pre-processing
- Train
- Evaluate

#### 1. Import Libray

In [2]:
import logging
import itertools

import numpy as np
import gensim

logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
#jupyter sometimes messes up the Logging setup:resore it
logging.root.level = logging.INFO

#return the first 'n' elements of the streatm, in here we want first 10 lines
def head(stream, n=10):
    return list(itertools.islice(stream, n))



change wikipedia dump to corpus

In [196]:
import smart_open
import gensim.corpora
import requests
from bs4 import BeautifulSoup

from gensim.utils import simple_preprocess, file_or_filename
#from gensim.corpora import WikiCorpus
from gensim.corpora.wikicorpus import extract_pages, filter_wiki
from gensim.parsing.preprocessing import STOPWORDS

In [62]:
#we splist sentence into words and test wheather it is in STOPWORDS, 
#if not in STOPWORDS, we tokenize this word into word frequence with its appear time
#simple_preprocess also transfer into lowercase 
def tokenize(text):
    return [token for token in simple_preprocess(text, min_len=2, max_len=20) if token not in STOPWORDS]

In [104]:
#transfore dumpu_file into (title, token) tuple to output

def iter_wiki(dump_file, ignore_str):
    idx = 0
    #if the sentence start from this word, we think it is not that useful, we eliminate it in preprocess 
    ignore_namespaces = ignore_str.split()
    #driectly use extract_pages from dump files. The yeild is tuple of (str or None, str, str) – Title, text and page id.
    for title, text, pageid in extract_pages(file_or_filename(dump_file)):
        #filter_wiki is used to extract text from tuple, output is str
        text_filter = filter_wiki(text)
        #get token from frome text_filter
        tokens = tokenize(text_filter)
        #ignore short articles and various meta-article
        if len(tokens) < 50 or any(title.startswith(ns+':') for ns in ignore_namespaces):
            continue
        yield title, tokens
    
    

In [83]:
"""

from nltk.tokenize import RegexpTokenizer
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

def nltk_wiki(dump_file, ignore_str):
    idx = 0
    docs= (tokens for _, tokens in iter_wiki('./data/simplewiki-20140623-pages-articles.xml.bz2'))
    #if the sentence start from this word, we think it is not that useful, we eliminate it in preprocess 
    ignore_namespaces = ignore_str.split()
    #driectly use extract_pages from dump files. The yeild is tuple of (str or None, str, str) – Title, text and page id.
    for title, text, pageid in extract_pages(file_or_filename(dump_file)):
        #filter_wiki is used to extract text from tuple, output is str
        #text_filter = filter_wiki(text)
        #get token from frome text_filter
        #tokens = tokenize(text_filter)
        #**********************************************************
        #i just want to use an alternative way to tokenize my raw material
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.
        idx+=1
        print(idx)
        
    
    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 1] for doc in docs]

    #We use the WordNet lemmatizer from NLTK. A lemmatizer is preferred over a stemmer in this case because it produces more readable words. 
    #Output that is easy to read is very desirable in topic modelling.
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]
    yield docs
"""

'\n\nfrom nltk.tokenize import RegexpTokenizer\n# Lemmatize the documents.\nfrom nltk.stem.wordnet import WordNetLemmatizer\n\ndef nltk_wiki(dump_file, ignore_str):\n    idx = 0\n    docs=[]\n    #if the sentence start from this word, we think it is not that useful, we eliminate it in preprocess \n    ignore_namespaces = ignore_str.split()\n    #driectly use extract_pages from dump files. The yeild is tuple of (str or None, str, str) – Title, text and page id.\n    for title, text, pageid in extract_pages(file_or_filename(dump_file)):\n        #filter_wiki is used to extract text from tuple, output is str\n        #text_filter = filter_wiki(text)\n        #get token from frome text_filter\n        #tokens = tokenize(text_filter)\n        #**********************************************************\n        #i just want to use an alternative way to tokenize my raw material\n        docs[idx] = docs[idx].lower()  # Convert to lowercase.\n        docs[idx] = tokenizer.tokenize(docs[idx])  

In [108]:
# Create a dictionary representation of the documents.
from gensim.corpora import Dictionary
def dictionaries(dump_file,ignore_str):
    doc_stream = (tokens for _, tokens in iter_wiki(dump_file,ignore_str))
    %time id2word_wiki = Dictionary(doc_stream)
    id2word_wiki.filter_extremes(no_below=20, no_above=0.1)
    print(id2word_wiki)
    yield id2word_wiki

#### 3.Vectorization

In [150]:
#parse the first 'clip_docs' Wikipedia documents form file 'dump_file'
#Yield each document in turn, as as list of tokens(unicode strings)
class WikiCorpus(object):
    def __init__(self, dump_file, dictionary,  ignore_str, clip_docs):
        self.dump_file = dump_file
        self.dictionary = dictionary
        self.ignore_str = ignore_str
        self.clip_docs =clip_docs
        
    
    def __iter__(self):
        self.titles = []
        for title, tokens in itertools.islice(iter_wiki(self.dump_file, self.ignore_str), self.clip_docs):
            self.titles.append(title)
            yield self.dictionary.doc2bow(tokens)
            
    def __len__(self):
        return self.clip_docs
        
        

#### 4.Convert html to text

In [224]:
import requests
from bs4 import BeautifulSoup

def url_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    test_tokens = tokenize(response.text)
    bow_test_doc = id2word_wiki.doc2bow(test_tokens)
    return print("This {} link have topic possibility : \n{}".format(url, lda_model.get_document_topics(bow_test_doc)))

#### 5.Main Funciton

In [106]:
if __name__ == '__main__': 
    #these pages are meta-data of wikipedia, which don't contain so much info. We shold ignore these pages
    ignore_str = 'Wikipedia Category File Portal template MediaWiki User Help Book Draft'
    source = 'D:/Downloads/31_simplewiki-20200301-pages-articles.xml.bz2'
    stream = iter_wiki(source,ignore_str)
    #word2vector = dictionaries(source,ignore_str)
    
    
    
    
    #for dmenstration, we only print first 8 line with first 10 words
    for title, tokens in itertools.islice(iter_wiki(source, ignore_str),8):
        print(title, tokens[:10])
        


INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(163649 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #20000 to Dictionary(241250 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #30000 to Dictionary(302107 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #40000 to Dictionary(365107 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #50000 to Dictionary(429710 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #60000 to Dictionary(463634 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #70000 to Dictionary(522621 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : b

Wall time: 6min 54s
Dictionary(34462 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
April ['april', 'fourth', 'month', 'year', 'comes', 'march', 'months', 'days', 'april', 'begins']
August ['august', 'aug', 'eighth', 'month', 'year', 'gregorian', 'calendar', 'coming', 'july', 'september']
Art ['painting', 'renoir', 'work', 'art', 'art', 'creative', 'activity', 'expresses', 'imaginative', 'technical']
A ['writing', 'cursive', 'letter', 'english', 'alphabet', 'small', 'letter', 'lower', 'case', 'vowel']
Air ['air', 'fan', 'air', 'air', 'earth', 'atmosphere', 'air', 'mixture', 'gases', 'dust']
Autonomous communities of Spain ['spain', 'divided', 'parts', 'called', 'autonomous', 'communities', 'autonomous', 'means', 'autonomous', 'communities']
Alan Turing ['statue', 'alan', 'turing', 'turing', 'idea', 'bombe', 'mechanical', 'details', 'added', 'built']
Alanis Morissette ['alanis', 'nadine', 'morissette', 'born', 'june', 'grammy', 'award', 'winning', 'can

In [152]:
"""
#create a stream of bag-of-words vectors
wiki_corps = WikiCorpus(source,word2vector,ignore_str,None)
#store all those bag-of-words vectors into a file, so we don't have to parse the bzipped Wikipedia XML every time over and over:
%time gensim.corpora.MmCorpus.serialize('D:/Downloads/35_wiki_bow_0317.mm',wiki_corps)
"""

INFO : storing corpus in Matrix Market format to D:/Downloads/35_wiki_bow_0317.mm
INFO : saving sparse matrix to D:/Downloads/35_wiki_bow_0317.mm
INFO : PROGRESS: saving document #0
INFO : PROGRESS: saving document #1000
INFO : PROGRESS: saving document #2000
INFO : PROGRESS: saving document #3000
INFO : PROGRESS: saving document #4000
INFO : PROGRESS: saving document #5000
INFO : PROGRESS: saving document #6000
INFO : PROGRESS: saving document #7000
INFO : PROGRESS: saving document #8000
INFO : PROGRESS: saving document #9000
INFO : PROGRESS: saving document #10000
INFO : PROGRESS: saving document #11000
INFO : PROGRESS: saving document #12000
INFO : PROGRESS: saving document #13000
INFO : PROGRESS: saving document #14000
INFO : PROGRESS: saving document #15000
INFO : PROGRESS: saving document #16000
INFO : PROGRESS: saving document #17000
INFO : PROGRESS: saving document #18000
INFO : PROGRESS: saving document #19000
INFO : PROGRESS: saving document #20000
INFO : PROGRESS: saving doc

Wall time: 7min 41s


In [154]:
#test for show tokeniza resule
vector = next(iter(wiki_corps))
print(vector)

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1), (9, 2), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 5), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 4), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 3), (35, 3), (36, 1), (37, 1), (38, 2), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 5), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 10), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 2), (70, 2), (71, 1), (72, 2), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 2), (85, 1), (86, 2), (87, 1), (88, 2), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 2), (96, 1), (97, 2), (98, 2), (99, 2), (100, 4), (101, 2), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 2), (108, 1), (109, 1), (110, 1)

In [157]:
#next time we can use mm_corpurs form file
mm_corpus = gensim.corpora.MmCorpus('D:/Downloads/35_wiki_bow_0317.mm')
print(mm_corpus)

INFO : loaded corpus index from D:/Downloads/35_wiki_bow_0317.mm.index
INFO : initializing cython corpus reader from D:/Downloads/35_wiki_bow_0317.mm
INFO : accepted corpus with 74654 documents, 34462 features, 7169754 non-zero entries


MmCorpus(74654 documents, 34462 features, 7169754 non-zero entries)


mm_corpus now contains exactly the same bag-of-words vectors as wiki_corpus before, but they are backed by the .mm file, rather than extracted on the fly from the xml.bz2 files

In [158]:
print(next(iter(mm_corpus)))

[(0, 1.0), (1, 2.0), (2, 1.0), (3, 1.0), (4, 1.0), (5, 1.0), (6, 2.0), (7, 1.0), (8, 1.0), (9, 2.0), (10, 1.0), (11, 3.0), (12, 1.0), (13, 1.0), (14, 1.0), (15, 2.0), (16, 1.0), (17, 5.0), (18, 1.0), (19, 1.0), (20, 1.0), (21, 1.0), (22, 1.0), (23, 1.0), (24, 1.0), (25, 2.0), (26, 4.0), (27, 1.0), (28, 1.0), (29, 1.0), (30, 2.0), (31, 1.0), (32, 1.0), (33, 1.0), (34, 3.0), (35, 3.0), (36, 1.0), (37, 1.0), (38, 2.0), (39, 1.0), (40, 1.0), (41, 1.0), (42, 1.0), (43, 1.0), (44, 1.0), (45, 1.0), (46, 2.0), (47, 1.0), (48, 1.0), (49, 5.0), (50, 1.0), (51, 1.0), (52, 1.0), (53, 1.0), (54, 1.0), (55, 1.0), (56, 1.0), (57, 1.0), (58, 1.0), (59, 10.0), (60, 2.0), (61, 1.0), (62, 1.0), (63, 1.0), (64, 1.0), (65, 1.0), (66, 2.0), (67, 1.0), (68, 1.0), (69, 2.0), (70, 2.0), (71, 1.0), (72, 2.0), (73, 1.0), (74, 1.0), (75, 1.0), (76, 1.0), (77, 1.0), (78, 1.0), (79, 1.0), (80, 1.0), (81, 1.0), (82, 1.0), (83, 2.0), (84, 2.0), (85, 1.0), (86, 2.0), (87, 1.0), (88, 2.0), (89, 1.0), (90, 2.0), (91, 1.

#### 7.Train LDA Model

Topic modeling in gensim is realized via transformations. A transformation is something that takes a corpus and spits out another corpus on output, using corpus_out = transformation_object[corpus_in] syntax. What exactly happens in between is determined by what kind of transformation we will using---Latenet Dirchlet Allocation

In [164]:
from gensim.models import LdaModel

# Set training parameters.
#The number of requested latent topics to be extracted from the training corpus.
num_topics = 10
#Number of documents to be used in each training chunk.
chunksize = 2000

passes = 4

iterations = 400
#Log perplexity is estimated every that many updates. Setting this to one slows down training by ~2x.
# Don't evaluate model perplexity, takes too much time.
eval_every = None 

# Make a index to word dictionary.
# This is only to "load" the dictionary.
id2word = id2word_wiki
#use fewer documents during training, LDA is slow
clipped_corpus = gensim.utils.ClippedCorpus(mm_corpus, 4000)

%time lda_model = LdaModel( \
    corpus=clipped_corpus, \
    id2word=id2word, \
    chunksize=chunksize, \
    alpha='auto', \
    eta='auto', \
    iterations=iterations, \
    num_topics=num_topics, \
    passes=passes, \
    eval_every=eval_every \
)

INFO : using autotuned alpha, starting with [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 10 topics, 4 passes over the supplied corpus of 4000 documents, updating model once every 2000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #2000/4000
INFO : optimized alpha [0.060138345, 0.064541206, 0.07009296, 0.06771941, 0.06987323, 0.05989014, 0.058921017, 0.057302512, 0.064253375, 0.060070235]
INFO : merging changes from 2000 documents into a model of 4000 documents
INFO : topic #7 (0.057): 0.007*"member" + 0.005*"republic" + 0.004*"singer" + 0.003*"actress" + 0.003*"politician" + 0.003*"british" + 0.003*"president" + 0.003*"french" + 0.003*"great" + 0.003*"league"
INFO : topic #6 (0.059): 0.006*"river" + 0.004*"league" + 0.003*"energy" + 0.003*"water" + 0.003*"premier" + 0.003*"cells" + 0.003*"j

INFO : merging changes from 2000 documents into a model of 4000 documents
INFO : topic #0 (0.049): 0.005*"bc" + 0.005*"team" + 0.005*"player" + 0.004*"ball" + 0.004*"game" + 0.004*"words" + 0.004*"players" + 0.004*"series" + 0.004*"play" + 0.004*"language"
INFO : topic #6 (0.050): 0.010*"river" + 0.010*"league" + 0.007*"energy" + 0.006*"water" + 0.005*"jpg" + 0.005*"premier" + 0.004*"cells" + 0.004*"division" + 0.004*"bridge" + 0.004*"file"
INFO : topic #1 (0.072): 0.006*"water" + 0.005*"person" + 0.004*"body" + 0.004*"things" + 0.004*"example" + 0.004*"food" + 0.003*"common" + 0.003*"types" + 0.003*"light" + 0.003*"animals"
INFO : topic #2 (0.073): 0.010*"music" + 0.004*"band" + 0.004*"movie" + 0.004*"album" + 0.003*"award" + 0.003*"numbers" + 0.003*"movies" + 0.003*"rock" + 0.003*"example" + 0.003*"love"
INFO : topic #3 (0.076): 0.013*"actor" + 0.011*"politician" + 0.011*"singer" + 0.011*"actress" + 0.010*"german" + 0.009*"footballer" + 0.009*"french" + 0.009*"player" + 0.008*"britis

Wall time: 36.8 s


In [176]:
#print a few most important words for each LDA topic
_ = lda_model.print_topics(num_topics=20, num_words=10)

INFO : topic #0 (0.050): 0.006*"player" + 0.005*"game" + 0.005*"team" + 0.005*"players" + 0.005*"series" + 0.004*"play" + 0.004*"ball" + 0.004*"television" + 0.004*"language" + 0.004*"played"
INFO : topic #1 (0.073): 0.006*"person" + 0.005*"water" + 0.005*"body" + 0.004*"things" + 0.004*"food" + 0.003*"example" + 0.003*"blood" + 0.003*"common" + 0.003*"types" + 0.003*"animals"
INFO : topic #2 (0.078): 0.011*"music" + 0.006*"band" + 0.006*"album" + 0.005*"movie" + 0.004*"rock" + 0.003*"song" + 0.003*"songs" + 0.003*"love" + 0.003*"award" + 0.003*"released"
INFO : topic #3 (0.082): 0.013*"actor" + 0.012*"politician" + 0.011*"singer" + 0.011*"actress" + 0.011*"german" + 0.010*"footballer" + 0.009*"french" + 0.009*"player" + 0.009*"writer" + 0.009*"british"
INFO : topic #4 (0.058): 0.016*"rgb" + 0.016*"hex" + 0.009*"color" + 0.005*"sea" + 0.005*"blue" + 0.004*"green" + 0.004*"red" + 0.004*"country" + 0.004*"pink" + 0.004*"purple"
INFO : topic #5 (0.069): 0.007*"country" + 0.005*"countries"

In [167]:
#Get the topics with the highest coherence score the coherence for each topic.
top_topics = lda_model.top_topics(clipped_corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

INFO : CorpusAccumulator accumulated stats from 1000 documents
INFO : CorpusAccumulator accumulated stats from 2000 documents
INFO : CorpusAccumulator accumulated stats from 3000 documents
INFO : CorpusAccumulator accumulated stats from 4000 documents


Average topic coherence: -1.9080.
[([(0.013439952, 'actor'),
   (0.012421263, 'politician'),
   (0.011165031, 'singer'),
   (0.010921399, 'actress'),
   (0.010623863, 'german'),
   (0.010118966, 'footballer'),
   (0.009358997, 'french'),
   (0.009270418, 'player'),
   (0.008643068, 'writer'),
   (0.008608789, 'british'),
   (0.0067787846, 'italian'),
   (0.006636178, 'president'),
   (0.004791825, 'composer'),
   (0.004776608, 'musician'),
   (0.004631176, 'minister'),
   (0.0045789587, 'canadian'),
   (0.004396165, 'russian'),
   (0.0041882666, 'prime'),
   (0.00416974, 'director'),
   (0.0039325184, 'japanese')],
  -0.3364554973149783),
 ([(0.009199669, 'republic'),
   (0.0072313575, 'member'),
   (0.0060036173, 'great'),
   (0.005561149, 'island'),
   (0.004916716, 'kingdom'),
   (0.0048644454, 'islands'),
   (0.004329085, 'york'),
   (0.004258775, 'president'),
   (0.0042336173, 'country'),
   (0.0042253393, 'british'),
   (0.0040855804, 'france'),
   (0.004038063, 'french'),
   (0

#### 8.Evaluation

In [230]:
url_1 = 'https://en.wikipedia.org/wiki/Jean_Baptiste_Fran%C3%A7ois_Pitra'
url_2 = 'https://en.wikipedia.org/wiki/Richard_Speiser'
url_3 = 'https://en.wikipedia.org/wiki/Siale_Piutau'

In [225]:
url_text(url_1)

This https://en.wikipedia.org/wiki/Jean_Baptiste_Fran%C3%A7ois_Pitra link have topic possibility : 
[(0, 0.07736982), (1, 0.01857841), (2, 0.09405043), (3, 0.0623004), (4, 0.03790229), (5, 0.032031614), (6, 0.081969656), (8, 0.063751094), (9, 0.5320338)]


In [226]:
url_text(url_2)

This https://en.wikipedia.org/wiki/Richard_Speiser link have topic possibility : 
[(0, 0.1151802), (1, 0.024985855), (2, 0.040951565), (3, 0.013642275), (4, 0.039383303), (6, 0.12123422), (8, 0.07866426), (9, 0.56592315)]


In [231]:
url_text(url_3)

This https://en.wikipedia.org/wiki/Siale_Piutau link have topic possibility : 
[(0, 0.18130021), (2, 0.0805679), (3, 0.019114021), (4, 0.11154415), (6, 0.07292845), (8, 0.086495854), (9, 0.44282055)]


In [229]:
url_test = 'https://en.wikipedia.org/wiki/Siale_Piutau'
response = requests.get(url_test)
soup = BeautifulSoup(response.text, 'html.parser')
#print(response.content)
#print(soup)
# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out
    
test_tokens = tokenize(response.text)
print(type(test_tokens))
#test_doc = preprocess_text(test_tokens)
#bow_vector_1 = id2word_wiki.doc2bow(tokenize(text))
bow_test_doc = id2word_wiki.doc2bow(test_tokens)

print("This {} link have topic possibility : \n{}".format(url_test, lda_model.get_document_topics(bow_test_doc)))

<class 'list'>
This https://en.wikipedia.org/wiki/Siale_Piutau link have topic possibility : 
[(0, 0.18130015), (2, 0.08056819), (3, 0.019114044), (4, 0.11154427), (6, 0.072928496), (8, 0.0864959), (9, 0.44282058)]


In [177]:
%time tfidf_model = gensim.models.TfidfModel(mm_corpus, id2word=id2word_wiki)

INFO : collecting document frequencies
INFO : PROGRESS: processing document #0
INFO : PROGRESS: processing document #10000
INFO : PROGRESS: processing document #20000
INFO : PROGRESS: processing document #30000
INFO : PROGRESS: processing document #40000
INFO : PROGRESS: processing document #50000
INFO : PROGRESS: processing document #60000
INFO : PROGRESS: processing document #70000
INFO : calculating IDF weights for 74654 documents and 34462 features (7169754 matrix non-zeros)


Wall time: 10.1 s


In [178]:
#the LSI transformation goes from a space of high dimensionality (TFIDF, tens of thousands) in to a space of low dimensionality(200)
#For this reason it can also seen as dimensinality reduction
%time lsi_model = gensim.models.LsiModel(tfidf_model[mm_corpus], id2word=id2word_wiki,num_topics=200)

INFO : using serial LSI version on this node
INFO : updating model with new documents
INFO : preparing a new chunk of documents
INFO : using 100 extra samples and 2 power iterations
INFO : 1st phase: constructing (34462, 300) action matrix
INFO : orthonormalizing (34462, 300) action matrix
INFO : 2nd phase: running dense svd on (300, 20000) matrix
INFO : computing the final decomposition
INFO : keeping 200 factors (discarding 15.066% of energy spectrum)
INFO : processed documents up to #20000
INFO : topic #0(15.852): -0.239*"footballer" + -0.223*"politician" + -0.218*"actor" + -0.200*"actress" + -0.188*"german" + -0.183*"singer" + -0.162*"french" + -0.159*"writer" + -0.154*"player" + -0.136*"british"
INFO : topic #1(10.687): -0.197*"footballer" + -0.176*"politician" + -0.154*"actor" + 0.150*"music" + -0.146*"actress" + -0.116*"singer" + -0.104*"writer" + -0.089*"player" + 0.082*"band" + -0.079*"italian"
INFO : topic #2(8.372): 0.297*"district" + -0.284*"music" + 0.249*"coat" + 0.235*"a

Wall time: 1min 12s


In [179]:
#we can store this "LSA via TFIDF via bag-of-words" corpus the same way again:
%time gensim.corpora.MmCorpus.serialize('D:/Downloads/36_wiki_tfidf.mm', tfidf_model[mm_corpus])
%time gensim.corpora.MmCorpus.serialize('D:/Downloads/37_wiki_lsa.mm', lsi_model[tfidf_model[mm_corpus]])

INFO : storing corpus in Matrix Market format to D:/Downloads/36_wiki_tfidf.mm
INFO : saving sparse matrix to D:/Downloads/36_wiki_tfidf.mm
INFO : PROGRESS: saving document #0
INFO : PROGRESS: saving document #1000
INFO : PROGRESS: saving document #2000
INFO : PROGRESS: saving document #3000
INFO : PROGRESS: saving document #4000
INFO : PROGRESS: saving document #5000
INFO : PROGRESS: saving document #6000
INFO : PROGRESS: saving document #7000
INFO : PROGRESS: saving document #8000
INFO : PROGRESS: saving document #9000
INFO : PROGRESS: saving document #10000
INFO : PROGRESS: saving document #11000
INFO : PROGRESS: saving document #12000
INFO : PROGRESS: saving document #13000
INFO : PROGRESS: saving document #14000
INFO : PROGRESS: saving document #15000
INFO : PROGRESS: saving document #16000
INFO : PROGRESS: saving document #17000
INFO : PROGRESS: saving document #18000
INFO : PROGRESS: saving document #19000
INFO : PROGRESS: saving document #20000
INFO : PROGRESS: saving document 

Wall time: 46.4 s


INFO : PROGRESS: saving document #0
INFO : PROGRESS: saving document #1000
INFO : PROGRESS: saving document #2000
INFO : PROGRESS: saving document #3000
INFO : PROGRESS: saving document #4000
INFO : PROGRESS: saving document #5000
INFO : PROGRESS: saving document #6000
INFO : PROGRESS: saving document #7000
INFO : PROGRESS: saving document #8000
INFO : PROGRESS: saving document #9000
INFO : PROGRESS: saving document #10000
INFO : PROGRESS: saving document #11000
INFO : PROGRESS: saving document #12000
INFO : PROGRESS: saving document #13000
INFO : PROGRESS: saving document #14000
INFO : PROGRESS: saving document #15000
INFO : PROGRESS: saving document #16000
INFO : PROGRESS: saving document #17000
INFO : PROGRESS: saving document #18000
INFO : PROGRESS: saving document #19000
INFO : PROGRESS: saving document #20000
INFO : PROGRESS: saving document #21000
INFO : PROGRESS: saving document #22000
INFO : PROGRESS: saving document #23000
INFO : PROGRESS: saving document #24000
INFO : PROGRE

Wall time: 1min 23s


In [182]:
text = 'A blood cell, also called a hematocyte, is a cell produced by hematopoiesis and normally found in blood'

#transform text into the bag-of-words space
bow_vector_1 = id2word_wiki.doc2bow(tokenize(text))
print([(id2word_wiki[id], count) for id, count in bow_vector_1])

[('normally', 1), ('blood', 2), ('produced', 1), ('cell', 2)]


In [185]:
#transfrom into LDA space
lda_vector_1 = lda_model[bow_vector_1]
print(lda_vector_1)
print(lda_model.print_topic(max(lda_vector_1,key=lambda item:item[1])[0]))

[(1, 0.91577274), (2, 0.0117065795), (3, 0.012393454), (5, 0.010448696)]
0.006*"person" + 0.005*"water" + 0.005*"body" + 0.004*"things" + 0.004*"food" + 0.003*"example" + 0.003*"blood" + 0.003*"common" + 0.003*"types" + 0.003*"animals"


In [ ]:

# store all trained models to disk
lda_model.save('./data/lda_wiki.model')
lsi_model.save('./data/lsi_wiki.model')
tfidf_model.save('./data/tfidf_wiki.model')
id2word_wiki.save('./data/wiki.dictionary')
# load the same model back; the result is equal to `lda_model`
same_lda_model = gensim.models.LdaModel.load('./data/lda_wiki.model')

In [89]:
doc_stream = (tokens for _, tokens in iter_wiki('D:/Downloads/31_simplewiki-20200301-pages-articles.xml.bz2', \
                                                'Wikipedia Category File Portal template MediaWiki User Help Book Draft'))
%time id2word_wiki = gensim.corpora.Dictionary(doc_stream)

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(163649 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #20000 to Dictionary(241250 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #30000 to Dictionary(302107 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #40000 to Dictionary(365107 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #50000 to Dictionary(429710 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #60000 to Dictionary(463634 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : adding document #70000 to Dictionary(522621 unique tokens: ['abdicated', 'abdicates', 'abraham', 'additionally', 'adolf']...)
INFO : b

Wall time: 8min 2s


In [113]:

doc = "A blood cell, also called a hematocyte, is a cell produced by hematopoiesis and normally found in blood."
doc_2='human human huma'
bow = id2word_wiki.doc2bow(tokenize(doc_2))
print(bow)

[(293, 2)]


In [120]:
print(id2word_wiki[34461])

isonames


In [43]:
extract_pages()

<function gensim.corpora.wikicorpus.extract_pages(f, filter_namespaces=False, filter_articles=None)>

In [56]:
file_or_filename(source)

In [51]:
title_1, text_1, pageid_1 = gensim.corpora.wikicorpus.extract_pages(smart_open(source))

TypeError: 'module' object is not callable

In [55]:
filter_func = gensim.corpora.wikicorpus.extract_pages(file_or_filename(source))

In [50]:
print(gensim.corpora.wikicorpus.filter_wiki(filter_func))

TypeError: decoding to str: need a bytes-like object, generator found

In [22]:
extract_pages

<function gensim.corpora.wikicorpus.extract_pages(f, filter_namespaces=False, filter_articles=None)>

#### 2. Defination